In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
import utm
from keras import backend as K
import math

def Dist_Error(pred,real):
    errors=np.abs(pred-real)
    result=[math.sqrt(math.pow(i[0],2)+math.pow(i[1],2))for i in errors]
    return result

def distance_loss(y_pred, y_true):
    return K.sqrt(K.mean(K.sum(K.square(y_pred-y_true), axis=-1)))

def median_absolute_deviation(y_pred, y_true):
    deviation = np.abs(y_pred-y_true)
    return np.mean(deviation, axis=0)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()


data = pd.read_csv('trainfix.csv')

X = [
    np.array([[mr[4:9],
              mr[9:14],
              mr[14:19],
              mr[19:24],
              mr[24:29],
              mr[29:34]]]).T for mr in data.values
]
X = np.array(X)
y = []
zone_number = 0
zone_letter = 0
for mr in data.values:
    lat, lon, zone_number, zone_letter = utm.from_latlon(mr[-4], mr[-5])
    y.append([lat, lon])
scaler = MinMaxScaler()
y = np.array(y)
scaler.fit(y)
y = scaler.transform(y)

def build_model():
    m = Sequential()
    m.add(Conv2D(32, kernel_size=2, activation='relu', input_shape=(5, 6, 1)))
    m.add(Conv2D(16, kernel_size=2, activation='relu'))
    m.add(Flatten())
    m.add(Dense(1024, activation='relu'))
    m.add(Dropout(0.2))
    m.add(Dense(512, activation='relu'))
    m.add(Dropout(0.2))
    m.add(Dense(512, activation='relu'))
    m.add(Dropout(0.2))
    m.add(Dense(256, activation='relu'))
    m.add(Dense(2))
    r = optimizers.Adam(lr=0.001,decay=0.00001)
    m.compile(optimizer=r, loss=distance_loss, metrics=['mse'])

    return m


history = LossHistory()
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
model = build_model()
model.fit(X, y,epochs=300, batch_size=64, callbacks=[history])
history.loss_plot('epoch')

Using TensorFlow backend.


Epoch 1/300
10047/10047 [==============================] - 9s 867us/step - loss: 0.2777 - mean_squared_error: 0.0411
Epoch 2/300
10047/10047 [==============================] - 1s 139us/step - loss: 0.2115 - mean_squared_error: 0.0226
Epoch 3/300
10047/10047 [==============================] - 1s 135us/step - loss: 0.1865 - mean_squared_error: 0.0175
Epoch 4/300
10047/10047 [==============================] - 1s 138us/step - loss: 0.1727 - mean_squared_error: 0.0150
Epoch 5/300
10047/10047 [==============================] - 1s 135us/step - loss: 0.1657 - mean_squared_error: 0.0139
Epoch 6/300
10047/10047 [==============================] - 1s 136us/step - loss: 0.1554 - mean_squared_error: 0.0122
Epoch 7/300
10047/10047 [==============================] - 1s 142us/step - loss: 0.1477 - mean_squared_error: 0.0110
Epoch 8/300
10047/10047 [==============================] - 1s 143us/step - loss: 0.1431 - mean_squared_error: 0.0104
Epoch 9/300
10047/10047 [==============================] - 1s 14

10047/10047 [==============================] - 1s 146us/step - loss: 0.0326 - mean_squared_error: 5.4947e-04
Epoch 138/300
10047/10047 [==============================] - 1s 140us/step - loss: 0.0312 - mean_squared_error: 5.0518e-04
Epoch 139/300
10047/10047 [==============================] - 1s 148us/step - loss: 0.0302 - mean_squared_error: 4.6936e-04
Epoch 140/300
10047/10047 [==============================] - 1s 144us/step - loss: 0.0313 - mean_squared_error: 5.0578e-04
Epoch 141/300
10047/10047 [==============================] - 2s 150us/step - loss: 0.0307 - mean_squared_error: 4.8095e-04
Epoch 142/300
10047/10047 [==============================] - 1s 144us/step - loss: 0.0325 - mean_squared_error: 5.4716e-04
Epoch 143/300
10047/10047 [==============================] - 1s 145us/step - loss: 0.0343 - mean_squared_error: 6.1953e-04
Epoch 144/300
10047/10047 [==============================] - 1s 143us/step - loss: 0.0306 - mean_squared_error: 4.7846e-04
Epoch 145/300
10047/10047 [===

10047/10047 [==============================] - 2s 150us/step - loss: 0.0234 - mean_squared_error: 2.8203e-04
Epoch 270/300
10047/10047 [==============================] - 2s 153us/step - loss: 0.0246 - mean_squared_error: 3.1803e-04
Epoch 271/300
10047/10047 [==============================] - 1s 144us/step - loss: 0.0231 - mean_squared_error: 2.7614e-04
Epoch 272/300
10047/10047 [==============================] - 1s 146us/step - loss: 0.0240 - mean_squared_error: 2.9958e-04
Epoch 273/300
10047/10047 [==============================] - 1s 145us/step - loss: 0.0239 - mean_squared_error: 3.0106e-04
Epoch 274/300
10047/10047 [==============================] - 1s 145us/step - loss: 0.0226 - mean_squared_error: 2.6237e-04
Epoch 275/300
10047/10047 [==============================] - 1s 146us/step - loss: 0.0213 - mean_squared_error: 2.3260e-04
Epoch 276/300
10047/10047 [==============================] - 1s 148us/step - loss: 0.0223 - mean_squared_error: 2.5812e-04
Epoch 277/300
10047/10047 [===

<Figure size 640x480 with 1 Axes>

In [ ]:
y_pred = model.predict(X_test)
y_real = scaler.inverse_transform(y_test)
y_pred = scaler.inverse_transform(y_pred)

In [ ]:
errors=Dist_Error(y_pred,y_real)
errors=np.array(errors)
print(np.median(errors))
print(np.mean(errors))
print(np.sort(errors)[int(len(errors)*0.9)])

In [ ]:
x = np.arange(0,np.max(errors))
error_y = [len(errors[errors < i])/len(errors) for i in x]
plt.plot(x, error_y)

In [3]:
data_test = pd.read_csv('testfix.csv')
X_test = [
    np.array([[mr[4:9],
              mr[9:14],
              mr[14:19],
              mr[19:24],
              mr[24:29],
              mr[29:34]]]).T for mr in data_test.values
]

X_test = np.array(X_test)
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(-1,2)
y_pred = scaler.inverse_transform(y_pred)
print(y_pred[0])

y_final = [[utm.to_latlon(i[0], i[1], zone_number=zone_number, zone_letter=zone_letter)[1],
            utm.to_latlon(i[0], i[1], zone_number=zone_number, zone_letter=zone_letter)[0]] for i in y_pred]
y_final = np.array(y_final)
df_pred = pd.DataFrame(data={'Longitude':y_final[:,0], 'Latitude':y_final[:,1]})
df_pred.to_csv('pred1.csv', index=False)

[ 329990.66 3463344.2 ]
